In [1]:
#library imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import talib
import warnings
from matplotlib.pyplot import figure
warnings.filterwarnings('ignore')

In [16]:
#data imports
volumes = pd.read_csv('Data/stock_volumes.csv', index_col = 'date', parse_dates = True)
prices = pd.read_csv('Data/stock_prices.csv', index_col = 'date', parse_dates = True)
info = pd.read_csv('Data/stock_info.csv')
sp_listings = pd.read_csv('Data/sp500_listings.csv', index_col = 'date', parse_dates = True)

## Data Cleaning

In [ ]:
for col in prices.columns:
    #select the stock
    stock = prices[[col]]
    #find delist date
    delist_date = sp_listings.where(sp_listings['stock'] == col).last_valid_index() + pd.DateOffset(1)
    #set all values after delist date to NaN
    stock.loc[delist_date:][col] = np.nan
    #replace in original dataframe
    prices[col] = stock[col]

In [58]:
prices

,905270,921795,904261,905261,916328,923024,936365,902355,912215,929813,...,9660J1,69568X,543755,77463M,29235J,131745,69487D,68157P,9110RA,292703
date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,22.75,12.63,30.00,29.61,3.8736,11.00,4.1250,24.50,58.00,19.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,22.56,12.57,31.50,29.37,3.8876,11.00,4.0000,24.50,57.75,18.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,22.38,12.41,32.25,28.89,3.8806,11.38,3.9375,24.63,57.88,18.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,21.81,12.60,32.13,28.41,3.8387,11.50,3.8125,24.75,57.50,18.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,22.25,12.49,32.13,29.13,3.8387,11.50,3.8125,25.00,57.75,19.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
drop_columns = []

for col in prices.columns:
    if prices[col].isna().value_counts()[1] == prices.shape[0]:
        drop_columns.append(col)

prices = prices.drop(columns = drop_columns)

In [51]:
prices_r = prices.copy(deep = True)
prices_r = prices_r.pct_change()*100

In [52]:
for col in prices_r.columns:
    #select the stock
    stock = prices_r[[col]]
    #find delist date
    delist_date = sp_listings.where(sp_listings['stock'] == col).last_valid_index() + pd.DateOffset(1)
    #set all values after delist date to NaN
    stock.loc[delist_date:][col] = np.nan
    #replace in original dataframe
    prices_r[col] = stock[col]

In [56]:
prices_r.to_pickle('./Data/returns.pkl')

## Data Preparation

In [72]:
#drop very first row
prices_r = prices_r.iloc[1:, :]

In [60]:
#sample stock
stock = '905270'